In [36]:
import sqlite3
import time
from faker import Faker
from faker_vehicle import VehicleProvider

# https://docs.python.org/3/library/sqlite3.html

In [62]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS vehicles")
table = """CREATE TABLE vehicles(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(255) NOT NULL,
    make VARCHAR(255) NOT NULL,
    model VARCHAR(255) NOT NULL,
    year TINYINT(4) NOT NULL,
    category VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2)
);
"""

cursor.execute(table)

In [63]:
fake = Faker()
fake.add_provider(VehicleProvider)

for x in range(0, 10000000):
    vehicle = fake.machine_object()
    input = (
        "{} {} ({})".format(vehicle['Make'], vehicle['Model'], vehicle['Year']), 
        vehicle['Make'], 
        vehicle['Model'], 
        vehicle['Year'], 
        vehicle['Category']
    )
    cursor.execute("""
        INSERT INTO vehicles(name, make, model, year, category) VALUES(?, ?, ?, ?, ?)
    """, input
    )
conn.commit()


In [64]:
def run_query(sql, name):
    start = time.time()

    res = cursor.execute(sql)
    res.fetchall()

    end = time.time()
    duration = str((end - start) * 1000)

    return {
        name,
        duration
    }
    

sql = "SELECT name, price FROM vehicles WHERE year = 2015"
q1 = run_query(sql, "vehicles_by_year_without_index")

cursor.execute("CREATE INDEX vehicles_make_model_year_idx ON vehicles(make, model, year);")
cursor.execute("CREATE INDEX vehicles_category_idx ON vehicles(category);")
cursor.execute("CREATE INDEX vehicles_make_idx ON vehicles(make);")
cursor.execute("CREATE INDEX vehicles_model_idx ON vehicles(model);")
cursor.execute("CREATE INDEX vehicles_year_idx ON vehicles(year);")

q2 = run_query(sql, "vehicles_by_year_with_index")

In [65]:
q1, q2

({'3112.1981143951416', 'vehicles_by_year_without_index'},
 {'955.6941986083984', 'vehicles_by_year_with_index'})

In [ ]:
"""
CREATE INDEX vehicles_category_index ON vehicles(make, model, year);
CREATE INDEX vehicles_category_index ON vehicles(category);
"""